# SQL Search (structured data) with LangChain

This example shows how to use the Python [LangChain](https://python.langchain.com/docs/get_started/introduction) library to run a natural language RAG search against structured data in a SQL database.

This leverages LangChain's [SQL Database Chains](https://python.langchain.com/docs/integrations/tools/sql_database/) to translate a natural language prompt into a SQL query. After getting a result, the LLM generates a natural language response.


## Set up the RAG workflow environment

In [ ]:
%%capture
!git clone https://github.com/VectorInstitute/rag_bootcamp /tmp/rag_bootcamp
!cp -ar /tmp/rag_bootcamp/sql_search/* ./

# Install langchain and LlamaIndex packages, filtering out version numbers
!grep -Eo '^(langchain|llama)[^=]*' \
    /tmp/rag_bootcamp/envs/rag_dataloaders/requirements.txt > \
    /tmp/requirements.txt
!pip3 install -r /tmp/requirements.txt

In [ ]:
import os

# TODO: fill in the two following lines
os.environ["OPENAI_API_KEY"] = "..."
os.environ["OPENAI_BASE_URL"] = "https://.../v1"

#### Import libraries

In [3]:
import warnings
warnings.filterwarnings('ignore')

In [ ]:
import os
import pandas as pd

from langchain_community.utilities import SQLDatabase
from langchain_experimental.sql import SQLDatabaseChain
from langchain_openai import ChatOpenAI

#### Load config files

In [5]:
GENERATOR_BASE_URL = os.environ["OPENAI_BASE_URL"]

OPENAI_API_KEY = os.environ["OPENAI_API_KEY"]

#### Choose LLM (no embedding model required)

In [6]:
GENERATOR_MODEL_NAME = "Meta-Llama-3.1-8B-Instruct"

## Set up your SQL data

In this example, we provide a financial dataset of term deposit applications from a Portuguese banking institution. A term deposit is a cash investment held at a financial institution, where money is invested for an agreed rate of interest over a fixed amount of time, or term. The dataset is available on [Kaggle](https://www.kaggle.com/datasets/prakharrathi25/banking-dataset-marketing-targets).

This dataset is stored in a SQLite database (`banking_term_deposits.db`) in the same folder as this notebook.

Let's look at a preview of the data.

In [7]:
df = pd.read_csv("banking_term_deposits.csv", delimiter=";")
df.head(10)

,age,job,marital,education,default,balance,housing,loan,contact,day,month,duration,campaign,pdays,previous,poutcome,y
0,58,management,married,tertiary,no,2143,yes,no,unknown,5,may,261,1,-1,0,unknown,no
1,44,technician,single,secondary,no,29,yes,no,unknown,5,may,151,1,-1,0,unknown,no
2,33,entrepreneur,married,secondary,no,2,yes,yes,unknown,5,may,76,1,-1,0,unknown,no
3,47,blue-collar,married,unknown,no,1506,yes,no,unknown,5,may,92,1,-1,0,unknown,no
4,33,unknown,single,unknown,no,1,no,no,unknown,5,may,198,1,-1,0,unknown,no
5,35,management,married,tertiary,no,231,yes,no,unknown,5,may,139,1,-1,0,unknown,no
6,28,management,single,tertiary,no,447,yes,yes,unknown,5,may,217,1,-1,0,unknown,no
7,42,entrepreneur,divorced,tertiary,yes,2,yes,no,unknown,5,may,380,1,-1,0,unknown,no
8,58,retired,married,primary,no,121,yes,no,unknown,5,may,50,1,-1,0,unknown,no
9,43,technician,single,secondary,no,593,yes,no,unknown,5,may,55,1,-1,0,unknown,no


# Start with a basic generation request without RAG augmentation

Let's start by asking Llama-3.1 a question about our SQL data which we don't expect it to know the answer to. For example:

*What is the average balance of all management jobs who applied for banking deposits?*

In [8]:
query = "What is the average balance of all management jobs who applied for banking deposits?"

## Now send the query to the open source model using KScope

In [9]:
llm = ChatOpenAI(
    model=GENERATOR_MODEL_NAME,
    temperature=0,
    base_url=GENERATOR_BASE_URL,
    api_key=OPENAI_API_KEY
)
message = [
    ("human", query),
]
try:
    result = llm.invoke(message)
    print(f"Result: \n\n{result.content}")
except Exception as err:
    if "Error code: 503" in err.message:
        print(f"The model {GENERATOR_MODEL_NAME} is not ready yet.")
    else:
        raise

Result: 

I can’t provide information on the average balance of management jobs who applied for banking deposits. Is there anything else I can help you with?


Naturally, without any context the model is unable to answer our question.

## Set up a database chain

Next, we want to set up a **database chain**. This is a tool that leverages the LLM to turn our question into a syntactically-correct SQL query.

In [10]:
db = SQLDatabase.from_uri("sqlite:///banking_term_deposits.db")
db_chain = SQLDatabaseChain.from_llm(llm, db, verbose=True)

Notably, let's take a look at the prompt that our database chain is about to ask the LLM.

In [11]:
print(db_chain.llm_chain.prompt.template)

You are a SQLite expert. Given an input question, first create a syntactically correct SQLite query to run, then look at the results of the query and return the answer to the input question.
Unless the user specifies in the question a specific number of examples to obtain, query for at most {top_k} results using the LIMIT clause as per SQLite. You can order the results to return the most informative data in the database.
Never query for all columns from a table. You must query only the columns that are needed to answer the question. Wrap each column name in double quotes (") to denote them as delimited identifiers.
Pay attention to use only the column names you can see in the tables below. Be careful to not query for columns that do not exist. Also, pay attention to which column is in which table.
Pay attention to use date('now') function to get the current date, if the question involves "today".

Use the following format:

Question: Question here
SQLQuery: SQL Query to run
SQLResult: 

## Send the chain request

A lot of magic happens in this next step. The following things happen in the background from the given line of code:
- The prompt above gets sent to the LLM (Llama-3.1), which returns a syntactically-correct SQL query.
- We send that query to our SQLite database and get some results.
- These results get sent back to the LLM along with our original query, which then returns a natural language response. Often times the response is just the basic answer, but sometimes the answer gets wrapped up in a longer text response.

In [12]:
result = db_chain.invoke(query)



> Entering new SQLDatabaseChain chain...
What is the average balance of all management jobs who applied for banking deposits?
SQLQuery:SELECT AVG("balance") FROM banking_term_deposits WHERE "job" = 'management'
SQLResult: [(1763.6168323112709,)]
Answer:Question: What is the average balance of all management jobs who applied for banking deposits?
SQLQuery: SELECT AVG("balance") FROM banking_term_deposits WHERE "job" = 'management'
> Finished chain.


#### Verify the answer using pandas

In [13]:
print(f"Correct answer using pandas: {df.loc[df['job']=='management', 'balance'].mean()}")

Correct answer using pandas: 1763.6168323112709


This matches with the answer (see ```SQLResult```) returned from the SQL chain above, but the model fails to provide the answer in the final result output. The prompt can be modified to fix this.

Try running the chain request again a few times. You'll probably see different responses each time, sometimes incorrect.